In [1]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
import numpy as np
import pickle

%matplotlib

Using matplotlib backend: TkAgg


# Plot times

Here we define a function to create a plot of setup time, average solve time and average integration time vs number of coupled DFN models, and a loglog plot of integration time vs number of states.

In [2]:
def plot_times(filename, title=None):
    "Plot times for a given filename"
    
    # unpickle times and make lists
    with open("data/" + filename, 'rb') as handle:
        times_and_states = pickle.load(handle)

    npts = list(times_and_states.keys())
    states = [times_and_states[i]["states"] for i in npts]
    setup_times = [times_and_states[i]["setup time"] for i in npts]
    solve_times = [times_and_states[i]["average solve time"] for i in npts]
    integration_times = [times_and_states[i]["average integration time"] for i in npts]

    # create figure
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    title = title or filename[:-4]
    
    # add times to plot
    ax[0].plot(npts, setup_times, "-o", label="Setup time")
    ax[0].plot(npts, solve_times, "-o", label="Solve time")
    ax[0].plot(npts, integration_times, "-o", label="Integration time")
    ax[1].loglog(states, integration_times, "-o")

    # add fit to states vs integration times 
    m, c = np.polyfit(np.log10(states), np.log10(integration_times), 1)
    ax[1].loglog(states, 10**c * states**m, "-")
    ax[1].text(1e3, 10, f"slope = {np.round(m,2)}")

    # add labels
    ax[0].set_xlabel("Number of coupled DFN models")
    ax[0].set_ylabel("Time [s]")            
    ax[0].legend()
    ax[1].set_xlabel("Number of states")
    ax[1].set_ylabel("Integration time [s]")        

    plt.suptitle(title)
    plt.tight_layout()
        

## Plot equipotential pair times 

In [3]:
plot_times("equipotential_pair.pkl")

## Plot isothermal equipotential pair times 

In [5]:
plot_times("isothermal_equipotential_pair.pkl")

## Plot potential pair times 

In [6]:
plot_times("potential_pair.pkl")

## Plot isothermal potential pair times 

In [7]:
plot_times("isothermal_potential_pair.pkl")

# Compare times for equipotential pair and potential pair model

Here we define a function to create a plot to compare setup times, average solve times and average integration times from one or more files.

In [8]:
def compare_times(filenames, labels=None):
    """Compare solve times for different files"""
    
    # create figure
    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    labels = labels or [filename[:-4] for filename in filenames]
    colors = iter(cm.tab10(np.linspace(0, 1, 10)))
    n = 0
    
    for filename, label in zip(filenames, labels):
        color = next(colors)
        n = n+1
        
        # unpickle times and make lists
        with open("data/" + filename, 'rb') as handle:
            times_and_states = pickle.load(handle)
            
        npts = list(times_and_states.keys())
        states = [times_and_states[i]["states"] for i in npts]
        setup_times = [times_and_states[i]["setup time"] for i in npts]
        solve_times = [times_and_states[i]["average solve time"] for i in npts]
        integration_times = [times_and_states[i]["average integration time"] for i in npts]

        # add times to plot
        ax[0, 0].plot(npts, setup_times, "-o", c=color)
        ax[0, 1].plot(npts, solve_times, "-o", c=color)
        ax[1, 0].plot(npts, integration_times, "-o", c=color)
        ax[1, 1].loglog(states, integration_times, "-o", label=label, c=color)

    # add labels
    ax[0, 0].set_xlabel("Number of coupled DFN models")
    ax[0, 0].set_ylabel("Setup time [s]")   
    ax[0, 1].set_xlabel("Number of coupled DFN models")
    ax[0, 1].set_ylabel("Solve time [s]")        
    ax[1, 0].set_xlabel("Number of coupled DFN models")
    ax[1, 0].set_ylabel("Integration time [s]")       
    ax[1, 1].set_xlabel("Number of states")
    ax[1, 1].set_ylabel("Integration time [s]")        
    if len(filenames) > 1:
        ax[1, 1].legend(loc="lower right")

    plt.tight_layout()
    
    return fig, ax

In [9]:
filenames = ["equipotential_pair.pkl", "potential_pair.pkl"]
_, ax = compare_times(filenames)